# **Extract Transform Load (ETL) Lab**


## Objectives

After completing this lab you will be able to:

*   Read CSV and JSON file types.
*   Extract data from the above file types.
*   Transform data.
*   Save the transformed data in a ready-to-load format which data engineers can use to load into an RDBMS.


Import the required modules and functions


In [18]:
import glob                         # this module helps in selecting files 
import pandas as pd                 # this module helps in processing CSV files
import xml.etree.ElementTree as ET  # this module helps in processing XML files.
from datetime import datetime

## Download Files


In [19]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/source.zip

--2022-07-10 16:09:25--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/source.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2707 (2.6K) [application/zip]
Saving to: ‘source.zip.1’

source.zip.1        100%[===================>]   2.64K  --.-KB/s    in 0s      

2022-07-10 16:09:26 (755 MB/s) - ‘source.zip.1’ saved [2707/2707]



## Unzip Files


In [23]:
!unzip source.zip

Archive:  source.zip
replace source3.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace source1.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace source2.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace source3.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace source1.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace source2.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace source1.xml? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace source2.xml? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace source3.xml? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


y## Set Paths


In [24]:
tmpfile    = "temp.tmp"               # file used to store all extracted data
logfile    = "logfile.txt"            # all event logs will be stored in this file
targetfile = "transformed_data.csv"   # file where transformed data is stored

## Extract


### CSV Extract Function


In [25]:
def extract_from_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process)
    return dataframe

### JSON Extract Function


In [26]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process,lines=True)
    return dataframe

### XML Extract Function


In [27]:
def extract_from_xml(file_to_process):
    dataframe = pd.DataFrame(columns=["name", "height", "weight"])
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    for person in root:
        name = person.find("name").text
        height = float(person.find("height").text)
        weight = float(person.find("weight").text)
        dataframe = dataframe.append({"name":name, "height":height, "weight":weight}, ignore_index=True)
    return dataframe

### Extract Function


In [28]:
def extract():
    extracted_data = pd.DataFrame(columns=['name','height','weight']) # create an empty data frame to hold extracted data
    
    #process all csv files
    for csvfile in glob.glob("*.csv"):
        extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
        
    #process all json files
    for jsonfile in glob.glob("*.json"):
        extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)
    
    #process all xml files
    for xmlfile in glob.glob("*.xml"):
        extracted_data = extracted_data.append(extract_from_xml(xmlfile), ignore_index=True)
        
    return extracted_data

## Transform


The transform function does the following tasks.

1.  Convert height which is in inches to millimeter
2.  Convert weight which is in pounds to kilograms


In [29]:
def transform(data):
        #Convert height which is in inches to millimeter
        #Convert the datatype of the column into float
        #data.height = data.height.astype(float)
        #Convert inches to meters and round off to two decimals(one inch is 0.0254 meters)
        data['height'] = round(data.height * 0.0254,2)
        
        #Convert weight which is in pounds to kilograms
        #Convert the datatype of the column into float
        #data.weight = data.weight.astype(float)
        #Convert pounds to kilograms and round off to two decimals(one pound is 0.45359237 kilograms)
        data['weight'] = round(data.weight * 0.45359237,2)
        return data

## Loading


In [30]:
def load(targetfile,data_to_load):
    data_to_load.to_csv(targetfile)  

## Logging


In [31]:
def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')

## Running ETL Process


In [32]:
log("ETL Job Started")

In [33]:
log("Extract phase Started")
extracted_data = extract()
log("Extract phase Ended")
extracted_data

,name,height,weight
0,alex,65.78,112.99
1,ajay,71.52,136.49
2,alice,69.40,153.03
3,ravi,68.22,142.34
4,joe,67.79,144.30
5,alex,65.78,112.99
6,ajay,71.52,136.49
7,alice,69.40,153.03
8,ravi,68.22,142.34
9,joe,67.79,144.30


In [34]:
log("Transform phase Started")
transformed_data = transform(extracted_data)
log("Transform phase Ended")
transformed_data 

,name,height,weight
0,alex,1.67,51.25
1,ajay,1.82,61.91
2,alice,1.76,69.41
3,ravi,1.73,64.56
4,joe,1.72,65.45
5,alex,1.67,51.25
6,ajay,1.82,61.91
7,alice,1.76,69.41
8,ravi,1.73,64.56
9,joe,1.72,65.45


In [35]:
log("Load phase Started")
load(targetfile,transformed_data)
log("Load phase Ended")

In [36]:
log("ETL Job Ended")

# Exercise 2



Using the example above complete the exercise below.


## Download Files


In [37]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip

--2022-07-10 16:11:29--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4249 (4.1K) [application/zip]
Saving to: ‘datasource.zip’

datasource.zip      100%[===================>]   4.15K  --.-KB/s    in 0s      

2022-07-10 16:11:30 (463 MB/s) - ‘datasource.zip’ saved [4249/4249]



## Unzip Files


In [38]:
!unzip datasource.zip -d dealership_data

Archive:  datasource.zip
  inflating: dealership_data/used_car_prices1.csv  
  inflating: dealership_data/used_car_prices2.csv  
  inflating: dealership_data/used_car_prices3.csv  
  inflating: dealership_data/used_car_prices1.json  
  inflating: dealership_data/used_car_prices2.json  
  inflating: dealership_data/used_car_prices3.json  
  inflating: dealership_data/used_car_prices1.xml  
  inflating: dealership_data/used_car_prices2.xml  
  inflating: dealership_data/used_car_prices3.xml  


## About the Data


The file `dealership_data` contains CSV, JSON, and XML files for used car data which contain features named `car_model`, `year_of_manufacture`, `price`, and `fuel`.


## Set Paths


In [39]:
tmpfile    = "dealership_temp.tmp"               # file used to store all extracted data
logfile    = "dealership_logfile.txt"            # all event logs will be stored in this file
targetfile = "dealership_transformed_data.csv"   # file where transformed data is stored

## Extract


### Question 1: CSV Extract Function


In [40]:
# Add the CSV extract function below


<details><summary>Click here for the solution</summary>

```
    
def extract_from_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process)
    return dataframe
```

</details>


### Question 2: JSON Extract Function


In [41]:
# Add the JSON extract function below


<details><summary>Click here for the solution</summary>

```
    
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process,lines=True)
    return dataframe
```

</details>


### Question 3: XML Extract Function


In [42]:
# Add the XML extract function below, it is the same as the xml extract function above but the column names need to be renamed.


<details><summary>Click here for the solution</summary>

```
    
def extract_from_xml(file_to_process):
    dataframe = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel'])
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    for person in root:
        car_model = person.find("car_model").text
        year_of_manufacture = int(person.find("year_of_manufacture").text)
        price = float(person.find("price").text)
        fuel = person.find("fuel").text
        dataframe = dataframe.append({"car_model":car_model, "year_of_manufacture":year_of_manufacture, "price":price, "fuel":fuel}, ignore_index=True)
    return dataframe
```

</details>


### Question 4: Extract Function

Call the specific extract functions you created above by replacing the `ADD_FUNCTION_CALL` with the proper function call.


In [43]:
def extract():
    extracted_data = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel']) # create an empty data frame to hold extracted data
    
    #process all csv files
    for csvfile in glob.glob("dealership_data/*.csv"):
        extracted_data = extracted_data.append('ADD_FUNCTION_CALL', ignore_index=True)
        
    #process all json files
    for jsonfile in glob.glob("dealership_data/*.json"):
        extracted_data = extracted_data.append('ADD_FUNCTION_CALL', ignore_index=True)
    
    #process all xml files
    for xmlfile in glob.glob("dealership_data/*.xml"):
        extracted_data = extracted_data.append('ADD_FUNCTION_CALL', ignore_index=True)
        
    return extracted_data

<details><summary>Click here for the solution</summary>

```
    
def extract():
    extracted_data = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel']) # create an empty data frame to hold extracted data
    
    #process all csv files
    for csvfile in glob.glob("dealership_data/*.csv"):
        extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
        
    #process all json files
    for jsonfile in glob.glob("dealership_data/*.json"):
        extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)
    
    #process all xml files
    for xmlfile in glob.glob("dealership_data/*.xml"):
        extracted_data = extracted_data.append(extract_from_xml(xmlfile), ignore_index=True)
        
    return extracted_data
```

</details>


## Transform


### Question 5: Transform

Round the `price` columns to 2 decimal places


In [44]:
# Add the transform function below


<details><summary>Click here for the solution</summary>

```

def transform(data):
        data['price'] = round(data.price, 2)
        return data
```

</details>


## Loading


### Question 6: Load


In [45]:
# Add the load function below


<details><summary>Click here for the solution</summary>

```

def load(targetfile,data_to_load):
    data_to_load.to_csv(targetfile)  
```

</details>


## Logging


### Question 7: Log

Make sure to change the name of the logfile to the one specified in the set paths section. Change the timestamp order to Hour-Minute-Second-Monthname-Day-Year.


In [46]:
# Add the log function below


<details><summary>Click here for the solution</summary>

```

def log(message):
    timestamp_format = '%H:%M:%S-%h-%d-%Y' #Hour-Minute-Second-MonthName-Day-Year
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("dealership_logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n') 
```

</details>


## Running ETL Process


### Question 8: ETL Process

Run all functions to extract, transform, and load the data. Make sure to log all events using the `log` function. Place your code under each comment.


In [47]:
# Log that you have started the ETL process


# Log that you have started the Extract step

# Call the Extract function

# Log that you have completed the Extract step


# Log that you have started the Transform step

# Call the Transform function

# Log that you have completed the Transform step


# Log that you have started the Load step

# Call the Load function

# Log that you have completed the Load step


# Log that you have completed the ETL process

<details><summary>Click here for the solution</summary>

```

log("ETL Job Started")

log("Extract phase Started")
extracted_data = extract()
log("Extract phase Ended")

log("Transform phase Started")
transformed_data = transform(extracted_data)
log("Transform phase Ended")

log("Load phase Started")
load(targetfile,transformed_data)
log("Load phase Ended")

log("ETL Job Ended")
```

</details>
